# 99 · Integración con Udacity Simulator

Orquesta el servidor `tools/sim_drive.py`:
1) Comprueba compatibilidad (modelo ↔ preprocesado).
2) Construye el comando.
3) Lanza y para el servidor desde aquí.

> Abre el simulador y entra en **Autonomous Mode** antes o después de lanzar el servidor.


In [6]:
# %%bash
# (Opcional) instala dependencias
# pip install python-socketio eventlet flask pillow opencv-python

In [7]:
from pathlib import Path
import sys, json, subprocess, shlex, time
from datetime import datetime

# === Raíz del proyecto ===
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.config import load_preset
from src.models import build_model
from src.datasets import ImageTransform

# === 1) Leer best_for_sim.json generado por el notebook de análisis ===
BEST_JSON = ROOT / "outputs" / "best_for_sim.json"
if not BEST_JSON.exists():
    raise FileNotFoundError(
        f"No existe {BEST_JSON}. "
        f"Ejecuta antes el notebook de análisis que genera best_for_sim.json."
    )

best = json.loads(BEST_JSON.read_text(encoding="utf-8"))

RUN_DIR_REL = best["run_dir"]
RUN_DIR_ABS = ROOT / "outputs" / RUN_DIR_REL

if not RUN_DIR_ABS.exists():
    raise FileNotFoundError(f"RUN_DIR_ABS no existe: {RUN_DIR_ABS}")

# Detectamos el preset con el que se entrenó ese run (std/accurate)
PRESET = best.get("preset", "accurate")
MODEL_NAME = "pilotnet_snn"   # el modelo que has usado en los runs CL
PORT = 4567

print("ROOT:", ROOT)
print("RUN_DIR_REL:", RUN_DIR_REL)
print("RUN_DIR_ABS:", RUN_DIR_ABS)
print("PRESET detectado:", PRESET)

# === 2) Buscar checkpoint en task_2_circuito2 de ese run ===
task2 = RUN_DIR_ABS / "task_2_circuito2"
if not task2.exists():
    raise FileNotFoundError(f"No existe carpeta task_2_circuito2 en {task2}")

ckpt_candidates = list(task2.glob("*.pt")) + list(task2.glob("*.pth"))
if not ckpt_candidates:
    raise FileNotFoundError(f"No se encontraron checkpoints (*.pt|*.pth) en {task2}")

def _pick_ckpt(files):
    best_like = [p for p in files if "best" in p.stem.lower()]
    if best_like:
        return max(best_like, key=lambda p: p.stat().st_mtime)
    return max(files, key=lambda p: p.stat().st_mtime)

CKPT = _pick_ckpt(ckpt_candidates)

print("CKPT seleccionado:", CKPT)

# === 3) Leer el preset para recuperar parámetros de datos/modelo ===
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)
DATA, MODEL = CFG["data"], CFG["model"]

ENCODER = DATA["encoder"]
T        = int(DATA["T"])
GAIN     = float(DATA["gain"])
W        = int(MODEL["img_w"])
H        = int(MODEL["img_h"])
TO_GRAY  = bool(MODEL["to_gray"])

print(f"[preset={PRESET}] model={MODEL_NAME} {W}x{H} gray={TO_GRAY} | enc={ENCODER} T={T} gain={GAIN}")


ROOT: /home/cesar/proyectos/TFM_SNN
RUN_DIR_REL: continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42
RUN_DIR_ABS: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42
PRESET detectado: std
CKPT seleccionado: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42/task_2_circuito2/best.pth
[preset=std] model=pilotnet_snn 200x66 gray=True | enc=rate T=18 gain=0.7


In [8]:
import torch

# Construye el modelo y carga el checkpoint para validar compatibilidad
tfm = ImageTransform(W, H, to_gray=TO_GRAY, crop_top=None)
model = build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)

state = torch.load(CKPT, map_location="cpu")

if isinstance(state, dict):
    if "state_dict" in state:
        sd = state["state_dict"]
    elif "model_state_dict" in state:
        sd = state["model_state_dict"]
    else:
        sd = state
else:
    sd = state

model.load_state_dict(sd)
model.eval()

print("OK: modelo cargado y listo para inferencia con ese checkpoint.")


OK: modelo cargado y listo para inferencia con ese checkpoint.


In [ ]:
# Construye el comando para sim_drive.py.
# Ojo: no forzamos encoder/T/gain/img_w/img_h por CLI,
# se cogen del preset. Así evitas inconsistencias.

cmd = f"""python tools/sim_drive.py \
  --ckpt {shlex.quote(str(CKPT))} \
  --model-name {MODEL_NAME} \
  --preset {PRESET} \
  --port {PORT} \
  --target-speed 12 \
  --crop-top 0
""".strip()

print("Comando:\n", cmd)

# Lanza el servidor como proceso hijo
proc = subprocess.Popen(cmd, shell=True, cwd=ROOT)
print("Servidor lanzado. PID =", proc.pid)


Comando:
 python tools/sim_drive.py   --ckpt /home/cesar/proyectos/TFM_SNN/outputs/continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42/task_2_circuito2/best.pth   --model-name pilotnet_snn   --preset std   --port 4567   --steer-scale 1.0   --target-speed 12   --crop-top 0
Servidor lanzado. PID = 118869


Ahora abre el **Udacity Simulator** y entra en **Autonomous Mode**.
Deberías ver cómo el coche comienza a moverse. Vuelve aquí para ver logs o parar el servidor.


In [10]:
import os, signal, time

time.sleep(2.0)  # pequeña espera para que arranque

if proc and (proc.poll() is None):
    print(f"Servidor VIVO (PID={proc.pid})")
else:
    print("Servidor NO está vivo (probablemente ha petado; mira la consola).")

# Para detenerlo manualmente:
# if proc and (proc.poll() is None):
#     os.kill(proc.pid, signal.SIGTERM)  # en Windows: proc.terminate()
#     time.sleep(1.0)
#     print("Servidor detenido.")


usage: Servidor de inferencia para el Udacity Simulator [-h] --ckpt CKPT
                                                        [--model-name {pilotnet_snn,pilotnet_ann,snn_vision}]
                                                        [--preset {fast,std,accurate}]
                                                        [--config CONFIG]
                                                        [--crop-top CROP_TOP]
                                                        [--encoder {None,rate,latency,raw,image}]
                                                        [--T T] [--gain GAIN]
                                                        [--img-w IMG_W]
                                                        [--img-h IMG_H]
                                                        [--rgb]
                                                        [--target-speed TARGET_SPEED]
                                                        [--kp KP] [--ki KI]
                                

Servidor NO está vivo (probablemente ha petado; mira la consola).


## Notas
- Este notebook solo orquesta la ejecución de `tools/sim_drive.py`.
- Si tienes errores de compatibilidad de dimensiones, revisa `img_w/img_h`, `to_gray` y `--crop-top`.
- Para curvas pronunciadas, baja `--target-speed` y ajusta `--kp/--kd`.
